<a href="https://colab.research.google.com/github/JosephGeorgeMtech/Finetuning_TransferLearning/blob/main/Fine_Tuning_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
%matplotlib inline

2.9.2


In [3]:
# load numpy array from npy file
from numpy import load

X_train_std = load('/content/drive/MyDrive/Transfer_Learning/Models/X_train_std.npy')
X_test_std = load('/content/drive/MyDrive/Transfer_Learning/Models/X_test_std.npy')

y_train = load('/content/drive/MyDrive/Transfer_Learning/Models/y_train.npy')
y_test = load('/content/drive/MyDrive/Transfer_Learning/Models/y_test.npy')

FileNotFoundError: ignored

In [ ]:
print("X_train_std_shape: {}".format(X_train_std.shape))
print("X_test_std_shape: {}".format(X_test_std.shape))

In [ ]:
model_FineTune = keras.models.load_model('/content/drive/MyDrive/Transfer_Learning/Models/01_Xception_TransferLearning_Best_Model.h5')

In [ ]:
model_FineTune.summary()

In [ ]:
model_FineTune.layers

In [ ]:
len(model_FineTune.layers)

In [ ]:
# Fine-tune the model from index 60 onwards

for layer in model_FineTune.layers[:60]:
  layer.trainable = False

for layer in model_FineTune.layers[60:]:
  layer.trainable = True

In [ ]:
model_FineTune.compile(loss='sparse_categorical_crossentropy', 
                 optimizer='adam', 
                 metrics=['accuracy'])

callbacks_FineTune = [            
             keras.callbacks.ModelCheckpoint("bestFT.h5",
                                             monitor='val_accuracy',
                                             save_weights_only=True,
                                             save_best_only=True)
]

history_FineTune = model_FineTune.fit(x = X_train_std, y = y_train, epochs=50,
                                      validation_split=0.1, batch_size=16, callbacks=callbacks_FineTune)

In [ ]:
keys = ['accuracy', 'val_accuracy']
progress = {k:v for k,v in history_FineTune.history.items() if k in keys}

import pandas as pd
pd.DataFrame(progress).plot()

plt.xlabel("epochs")
plt.ylabel("accuracy")

plt.grid(True)
plt.show()

In [ ]:
testLoss_FineTune, testAccuracy_FineTune = model_FineTune.evaluate(x = X_test_std, y = y_test)
print("Test-loss: %f, Test-accuracy: %f" % (testLoss_FineTune, testAccuracy_FineTune))

In [ ]:
model_FineTune.load_weights("bestFT.h5")

testLoss_FineTune, testAccuracy_FineTune = model_FineTune.evaluate(x = X_test_std, y = y_test)

print("Test-loss: %f, Test-accuracy: %f" % (testLoss_FineTune, testAccuracy_FineTune))

In [ ]:
cProba = model_FineTune.predict(X_test_std[:1])

print(cProba)

In [ ]:
cLabel = np.argmax(cProba, axis=-1) 

print(cLabel)

In [ ]:
print(y_test[:1])

In [ ]:
# Saves the best model obtained
model_FineTune.save('/content/drive/MyDrive/Models/02_Xception_FineTuned_Best_Model.h5')